In [1]:
!sudo apt-get install -y build-essential python-dev
!pip install gdrivefs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
build-essential is already the newest version (12.9).
python-dev-is-python2 is already the newest version (2.7.18-9).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.


In [2]:
!gdfstool auth_get_url

To authorize FUSE to use your Google Drive account, visit the following URL to produce an authorization code:

https://accounts.google.com/o/oauth2/auth?client_id=1037099463298-7b3a9u7g8q8irvemvc265c8uo65srrhv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&access_type=offline&response_type=code



In [7]:
!unzip ds2_images_extracted.zip

unzip:  cannot find or open ds2_images_extracted.zip, ds2_images_extracted.zip.zip or ds2_images_extracted.zip.ZIP.


In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

  Cloning https://github.com/openai/CLIP.git to /var/tmp/pip-req-build-q0rt7zf8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /var/tmp/pip-req-build-q0rt7zf8
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [2]:
import json
import os
from PIL import Image

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

In [5]:
print(device)

cuda:0


In [6]:
class image_title_dataset(Dataset):
    def __init__(self, list_image_path, list_txt):
        self.image_path = list_image_path
        self.title = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        try:
            image = preprocess(Image.open(self.image_path[idx]))
        except FileNotFoundError:
            print(f"File not found: {self.image_path[idx]}. Skipping...")
            return None, None

        title = self.title[idx]
        return image, title

In [7]:
def createDataLoader(json_path):
    with open(json_path, "r") as file:
        json_data = json.load(file)

    input_data = []
    for item in json_data:
        input_data.append(item)

    list_image_path = []
    list_txt = []
    for item in input_data:
        img_path = item['file_path']
        if os.path.exists(img_path):
            caption = item['labels'][0]
            list_image_path.append(img_path)
            list_txt.append(caption)
        else:
            print(f"File not found: {img_path}. Skipping...")

    dataset = image_title_dataset(list_image_path, list_txt)
    dataloader = DataLoader(dataset, batch_size=100, shuffle=True, drop_last=True)
    return dataloader

In [8]:
%cd ./freiburg_groceries_dataset_test_train/

/home/jupyter/freiburg_groceries_dataset_test_train


In [9]:
ls -l

total 696
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 BEANS/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 CAKE/
drwxr-xr-x 4 jupyter jupyter  20480 Mar 17 21:07 CANDY/
drwxr-xr-x 4 jupyter jupyter  12288 Mar 17 21:07 CEREAL/
drwxr-xr-x 4 jupyter jupyter  12288 Mar 17 21:07 CHIPS/
drwxr-xr-x 4 jupyter jupyter  20480 Mar 17 21:07 CHOCOLATE/
drwxr-xr-x 4 jupyter jupyter  16384 Mar 17 21:07 COFFEE/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 CORN/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 FISH/
drwxr-xr-x 5 jupyter jupyter   4096 Mar 24 16:17 FLOUR/
drwxr-xr-x 4 jupyter jupyter  12288 Mar 17 21:07 HONEY/
drwxr-xr-x 4 jupyter jupyter  12288 Mar 17 21:07 JAM/
drwxr-xr-x 4 jupyter jupyter  12288 Mar 17 21:07 JUICE/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 MILK/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 NUTS/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 17 21:07 OIL/
drwxr-xr-x 4 jupyter jupyter  12288 Mar 17 21:07 PASTA/
drwxr-xr-x 4 jupyter jupyter   4096 Mar 1

In [10]:
train_path = './train_data.json'
test_path = './test_data.json'

In [11]:
test_dataloader = createDataLoader(test_path)

In [12]:
train_dataloader = createDataLoader(train_path)

In [13]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()


if device == "cpu":
  model.float()

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

In [14]:
def train_model(num_epochs):
  for epoch in range(num_epochs):
      pbar = tqdm(train_dataloader, total=len(train_dataloader))
      for batch in pbar:
          optimizer.zero_grad()

          images,texts = batch

          images= images.to(device)
          texts = texts.to(device)

          logits_per_image, logits_per_text = model(images, texts)

          ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
          total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

          total_loss.backward()
          if device == "cpu":
              optimizer.step()
          else :
              convert_models_to_fp32(model)
              optimizer.step()
              clip.model.convert_weights(model)

          pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")

In [15]:
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            images, texts = batch
            images = images.to(device)
            texts = texts.to(device)

            logits_per_image, logits_per_text = model(images, texts)
            _, predicted = torch.max(logits_per_image + logits_per_text, 1)
            total += texts.size(0)
            correct += (predicted == torch.arange(len(images), dtype=torch.long, device=device)).sum().item()

    accuracy = correct / total
    return accuracy

In [53]:
!pip install torch

In [55]:
import torch
torch.__version__

'2.2.1+cu121'

In [56]:
!sudo apt-get update

Hit:1 https://deb.debian.org/debian bullseye InRelease
Hit:2 https://download.docker.com/linux/debian bullseye InRelease              
Get:3 https://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:4 https://nvidia.github.io/libnvidia-container/stable/debian10/amd64  InRelease [1484 B]
Hit:5 https://nvidia.github.io/nvidia-container-runtime/stable/debian10/amd64  InRelease
Get:6 https://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Hit:7 https://nvidia.github.io/nvidia-docker/debian10/amd64  InRelease     
Get:8 https://deb.debian.org/debian bullseye-backports InRelease [49.0 kB]
Hit:9 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Get:10 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease [5146 B]
Hit:11 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Get:12 https://packages.cloud.google.com/apt google-fast-socket InRelease [5015 B]
Get:13 https://deb.debian.org/debian-security 

In [65]:
!sudo apt install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package nvidia-cuda-toolkit is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'nvidia-cuda-toolkit' has no installation candidate


In [19]:
!pip install torch

In [20]:
import torch
>>> torch.__version__

'2.2.1+cu121'

In [21]:
train_model(1)

  0%|          | 0/40 [00:00<?, ?it/s]Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22query

RuntimeError: GET was unable to find an engine to execute this computation

In [ ]:
print(evaluate_model(model))